## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-04-03-34 +0000,nypost,Cedar Point’s ‘Siren’s Curse’ rollercoaster br...,https://nypost.com/2025/07/28/us-news/cedar-po...
1,2025-07-28-04-01-24 +0000,nyt,Last Soldiers of an Imperial Army Have a Warni...,https://www.nytimes.com/2025/07/28/world/asia/...
2,2025-07-28-04-01-22 +0000,nyt,Thai-Cambodian Conflict Tears at Remnants of a...,https://www.nytimes.com/2025/07/28/world/asia/...
3,2025-07-28-04-01-15 +0000,nyt,U.S. and China Meet as Trade Truce Nears Expir...,https://www.nytimes.com/2025/07/28/business/us...
4,2025-07-28-04-01-09 +0000,nyt,The Country Where 76% of Cars Sold Are Electric,https://www.nytimes.com/2025/07/28/business/ne...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
105,trump,28
336,gaza,15
397,aid,11
394,israel,10
33,trade,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
114,2025-07-27-17-13-12 +0000,nypost,Trump says ‘Israel is gonna have to make a dec...,https://nypost.com/2025/07/27/us-news/trump-sa...,72
110,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...,64
179,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...,61
79,2025-07-27-20-44-00 +0000,wsj,President Trump said he reached a trade agreem...,https://www.wsj.com/economy/trade/trump-says-h...,60
142,2025-07-27-12-10-56 +0000,nypost,Trump and Ursula von der Leyen to meet for hig...,https://nypost.com/2025/07/27/world-news/trump...,60


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
114,72,2025-07-27-17-13-12 +0000,nypost,Trump says ‘Israel is gonna have to make a dec...,https://nypost.com/2025/07/27/us-news/trump-sa...
102,40,2025-07-27-18-25-51 +0000,cbc,"U.S., EU agree to framework for trade deal tha...",https://www.cbc.ca/news/world/us-eu-trade-deal...
29,34,2025-07-28-00-23-00 +0000,nypost,Thousands of New Yorkers — including ‘Orange i...,https://nypost.com/2025/07/27/us-news/thousand...
179,29,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...
38,27,2025-07-27-23-44-06 +0000,nypost,"California mom-of-four Whisper Owen, baby go m...",https://nypost.com/2025/07/27/us-news/californ...
58,26,2025-07-27-22-21-33 +0000,nypost,Disturbing video shows woman knocked out cold ...,https://nypost.com/2025/07/27/us-news/disturbi...
80,24,2025-07-27-20-30-26 +0000,nypost,At least 3 killed and others injured in train ...,https://nypost.com/2025/07/27/world-news/at-le...
56,23,2025-07-27-22-24-26 +0000,nypost,Hundreds of Gen Zers attempt world record at l...,https://nypost.com/2025/07/27/us-news/hundreds...
72,23,2025-07-27-21-24-50 +0000,nyt,Suspect in Traverse City Walmart Stabbing Atta...,https://www.nytimes.com/2025/07/27/us/walmart-...
136,23,2025-07-27-13-09-41 +0000,nyt,What Will It Cost to Renovate Trump’s ‘Free’ A...,https://www.nytimes.com/2025/07/27/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
